In [1]:
#installing packages 

import numpy as np
import pandas as pd
from pandas_datareader import data as web
import pandas_datareader
import datetime
import matplotlib.pyplot as plt
from scipy import stats
%matplotlib inline

# Creating a Portfolio

In [2]:
# Creating the portfolio weights (4 weights)

portfolio_weights = np.array([0.30, 0.20, 0.25, 0.25])

In [3]:
# Creating the porftolio (4 variables)

In [5]:
start = datetime.datetime(2014, 1, 2)
end = datetime.datetime(2019, 1, 1)

tickers = ['NFLX', 'DIS', 'TSLA', 'AMZN']
stocks = pd.DataFrame()
for x in tickers:
    stocks[x] = web.DataReader(x, 'yahoo', start, end)['Close']


In [111]:
# Creating the return of the portfolio (not logarithmic mathematically correct rp = w1r1 + w2r2)

stocks_return = (stocks/stocks.shift(1))

In [112]:
# drop missing values

stocks_return.dropna().head()

,NFLX,DIS,TSLA,AMZN
Date,,,,
2014-01-03,1.000772,0.997902,0.996402,0.996155
2014-01-06,0.990278,0.996190,0.982883,0.992912
2014-01-07,0.944183,1.006858,1.016054,1.011178
2014-01-08,1.004389,0.985329,1.012855,1.009773
2014-01-09,0.988445,0.995746,0.975212,0.997736


In [113]:
weighted_returns_portfolio = stocks_return.mul(portfolio_weights, axis = 1)

weighted_returns_portfolio.head()

,NFLX,DIS,TSLA,AMZN
Date,,,,
2014-01-02,NaN,NaN,NaN,NaN
2014-01-03,0.300232,0.199580,0.249101,0.249039
2014-01-06,0.297083,0.199238,0.245721,0.248228
2014-01-07,0.283255,0.201372,0.254014,0.252794
2014-01-08,0.301317,0.197066,0.253214,0.252443


In [114]:
#Calculating the returns of the portfolio

stocks_return['Portfolio'] = weighted_returns_portfolio.sum(axis=1).dropna()

In [115]:
cumulative_returns = ((1+stocks_return["Portfolio"]).cumprod()-1)

In [116]:
#add risk free rate to the dataframe (1 month or 3 months only one available for the investor)

stocks_return['RF Rate'] = 0.0103

In [117]:
# calculate the exceess of the porfolio. 

stocks_return['excess'] = stocks_return['Portfolio'] - stocks_return['RF Rate']

In [118]:
stocks_return.head().dropna()


,NFLX,DIS,TSLA,AMZN,Portfolio,RF Rate,excess
Date,,,,,,,
2014-01-03,1.000772,0.997902,0.996402,0.996155,0.997951,0.0103,0.987651
2014-01-06,0.990278,0.996190,0.982883,0.992912,0.990270,0.0103,0.979970
2014-01-07,0.944183,1.006858,1.016054,1.011178,0.991435,0.0103,0.981135
2014-01-08,1.004389,0.985329,1.012855,1.009773,1.004039,0.0103,0.993739


In [136]:
start = datetime.datetime(2014, 1, 2)
end = datetime.datetime(2019, 1, 1)

stocks_return['Market'] = web.DataReader('SPY','yahoo',start,end)['Close']

stocks_return['Market'] = (stocks_return['Market']/stocks_return['Market'].shift(1))

stocks_return.head().dropna()

,NFLX,DIS,TSLA,AMZN,Portfolio,RF Rate,excess,Market
Date,,,,,,,,
2014-01-03,1.000772,0.997902,0.996402,0.996155,0.997951,0.0103,0.987651,0.999836
2014-01-06,0.990278,0.996190,0.982883,0.992912,0.990270,0.0103,0.979970,0.997102
2014-01-07,0.944183,1.006858,1.016054,1.011178,0.991435,0.0103,0.981135,1.006142
2014-01-08,1.004389,0.985329,1.012855,1.009773,1.004039,0.0103,0.993739,1.000218


In [139]:
stocks_return['excess market'] = stocks_return['Market'] - stocks_return['RF Rate']

stocks_return.head().dropna()

,NFLX,DIS,TSLA,AMZN,Portfolio,RF Rate,excess,Market,excess market
Date,,,,,,,,,
2014-01-03,1.000772,0.997902,0.996402,0.996155,0.997951,0.0103,0.987651,0.999836,0.989536
2014-01-06,0.990278,0.996190,0.982883,0.992912,0.990270,0.0103,0.979970,0.997102,0.986802
2014-01-07,0.944183,1.006858,1.016054,1.011178,0.991435,0.0103,0.981135,1.006142,0.995842
2014-01-08,1.004389,0.985329,1.012855,1.009773,1.004039,0.0103,0.993739,1.000218,0.989918


In [148]:
#calculating Beta - covariance matrix

covariance_matrix = stocks_return[['excess', 'excess market']].cov()
covariance_matrix

,excess,excess market
excess,0.001065,0.000091
excess market,0.000091,0.000070


In [149]:
covariance_coefficient = covariance_coefficient = covariance_matrix.iloc[0, 1]
covariance_coefficient

9.069614420327874e-05

In [151]:
variance_coefficient = stocks_return['excess market'].var()
variance_coefficient

6.987461563736775e-05

In [152]:
# Beta calculation

beta = covariance_coefficient/variance_coefficient
beta

1.2979841588534764

In [154]:
capm = stocks_return['RF Rate'] + beta*stocks_return['excess']
capm.tail()

Date
2018-12-26    1.406415
2018-12-27    1.287570
2018-12-28    1.319439
2018-12-31    1.322427
2019-01-02    1.279219
dtype: float64